In [1]:
import pandas as pd 
import numpy as np
import sqlalchemy
import pyodbc

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.window as w
import pyspark.sql.types as t

In [2]:
spark = SparkSession.builder.appName('TheVoice-ETL').getOrCreate()

In [3]:
# Load into dataframes
call_type_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\call_type.parquet')
countries_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\countries.parquet')
customer_mrr =          spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\customer.parquet')
customer_invoice_mrr =  spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\customer_invoice.parquet')
customer_lines_mrr =    spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\customer_lines.parquet')
opfileopp_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\pfileopp.parquet')
package_catalog_mrr =   spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\package_catalog.parquet')
usage_main_mrr =        spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\usage_main.parquet')
xxCountryType_mrr =     spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\parquet_files\xxCountryType.parquet')

In [4]:
# DimCallTypes staging

# variable set
price_per_minute = 0.5

# window for the key column
windowSpec_callType = w.Window.orderBy('call_type_code')

# final df 
DimCallTypes_stg = call_type_mrr.withColumn('KeyCallType', f.row_number().over(windowSpec_callType) + 1000)\
    .withColumnRenamed('call_type_code', 'DescCallTypeCode')\
    .withColumnRenamed('call_type_desc', 'DescCallType')\
    .withColumn('DescFullCallType', 
        f.concat_ws('-', f.col('DescCallTypeCode'), f.col('DescCallType')))\
    .withColumn('DescCallTypePriceCategory', f.when(f.col('priceperminuter') > price_per_minute, 'Discounted Price').otherwise('Normal Price'))\
    .withColumnRenamed('call_type', 'DescCallTypeCategory')\
    .select('KeyCallType', 'DescCallTypeCode', 'DescCallType', 'DescFullCallType', 'DescCallTypePriceCategory', 'DescCallTypeCategory')#.show()


In [5]:
# DimCountries staging

# select proper column and rename the country code column
xxCountryType_mrr = xxCountryType_mrr.select('COUNTRY_CODE', 'COUNTRY_PRE').withColumnRenamed('COUNTRY_CODE','COUNTRY_CODE2')

# final df
DimCountries_stg = countries_mrr.select('COUNTRY_CODE', 'REGION', 'AREA').join(
    xxCountryType_mrr,
    countries_mrr['COUNTRY_CODE'] == xxCountryType_mrr['COUNTRY_CODE2'],
    how='inner')\
    .withColumnRenamed('COUNTRY_PRE', 'KeyCountry')\
    .withColumnRenamed('COUNTRY_CODE', 'DescCountry')\
    .withColumnRenamed('REGION', 'DescRegion')\
    .withColumnRenamed('AREA', 'DescArea')\
    .select('KeyCountry', 'DescCountry', 'DescRegion', 'DescArea')#.show(5)

In [6]:
package_catalog_mrr.show(5)

+-----------+--------------------+----------+------+---------+--------------------+--------------------+--------------------+
|PACKAGE_NUM|          createdate|   enddate|status|pack_type|           pack_desc|         insert_date|         update_date|
+-----------+--------------------+----------+------+---------+--------------------+--------------------+--------------------+
|          1|2014-01-15 16:28:...|      NULL|     1|   FAMILY|pay much more get...|2014-01-15 16:28:...|2014-01-15 16:28:...|
|          2|2014-01-15 16:28:...|      NULL|     1|  FRIENDS|friends pay more ...|2014-01-15 16:28:...|2014-01-15 16:28:...|
|          3|2014-01-15 16:28:...|      NULL|     1| BUSINESS|business man pay ...|2014-01-15 16:28:...|2014-01-15 16:28:...|
|          4|2012-01-22 00:00:...|2015-01-23|     0| SOLDIERS|   Soldiers pay less|2012-01-22 00:00:...|2015-01-23 00:00:...|
+-----------+--------------------+----------+------+---------+--------------------+--------------------+--------------

In [11]:
# DimPackageCatalog staging
DimPackageCatalog_stg = package_catalog_mrr.withColumnRenamed('PACKAGE_NUM','KeyPackage')\
.withColumnRenamed('pack_desc', 'DescPackage')\
.withColumnRenamed('createdate', 'DatePackageCreation')\
.withColumnRenamed('enddate', 'DatePackageEnd')\
.withColumn('DescPackageStatus',
    f.when(f.col('status') == 1, 'Active').otherwise('Inactive'))\
.withColumn('CodePackageActivitiesDays', f.date_diff(f.col('DatePackageEnd'), f.col('DatePackageCreation')))
